# Parte 1: Utilidade

## Importar os pacotes necessários

In [ ]:
# importar os pacotes necessários para fazer análise
import pandas as pd, numpy as np
import os, re
import requests
import zipfile

## Definir funções para processar os dados

In [ ]:
# definir função que baixa os dados
def baixar_tse():
    URL = 'http://agencia.tse.jus.br/estatistica/sead/odsele/motivo_cassacao/motivo_cassacao_2016.zip'
    r = requests.get(URL)
    with open('../dados/motivo_cassacao2016.zip', 'wb') as arquivo:
        arquivo.write(r.content)
    return 'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

# definir função de descomprime arquivo do tse
def unzip_tse():
    with zipfile.ZipFile('../dados/motivo_cassacao2016.zip', 'r') as zip:
        zip.extractall('../dados')
    return 'Arquivo(s) descomprimido(s) com sucesso.'

# definir função para juntar os dados do tse
def juntar_tse():
    regex = re.compile(r'motivo.*csv$')
    juntar_tse.kwargs = {
        'engine': 'python', 'encoding': 'latin1', 'sep':';', 'quoting': 1,
        'dtype': str
    }
    arquivos = os.listdir('../dados')
    arquivos = list(filter(regex.search, arquivos))
    arquivos = [
        os.path.join(os.path.realpath('../dados'), arquivo) \
        for arquivo in arquivos
    ]
    dados = pd.concat(
        [pd.read_csv(arquivo, **juntar_tse.kwargs) for arquivo in arquivos],
        ignore_index=True
    )
    dados = dados[dados['DS_MOTIVO_CASSACAO'].notna()]
    print('Arquivo(s) juntado(s) com sucesso.')
    return dados

## Executar funções

In [ ]:
# executar função
# baixar_tse()

In [ ]:
# executar função
unzip_tse()

In [ ]:
# executar função e salvar o resultado no banco de dados chamado "candidaturas"
candidaturas = juntar_tse()

## Exibir dados preliminares

In [ ]:
# mostrar os dados
candidaturas.sample(10)

In [ ]:
# verificar os motivos para cassação
list(candidaturas['DS_MOTIVO_CASSACAO'].unique())

## Focar no Paraná

In [ ]:
# focar em 10 candidatos no paraná
amostra_pr = candidaturas[candidaturas['SG_UF'] == 'PR'].sample(10, random_state=67)

In [ ]:
# mostrar quais informações nós obtivemos
amostra_pr.columns.to_list()

In [ ]:
# juntar os motivos de cassação com outros dados dos candidatos?
candidatos_pr = pd.read_csv('../dados/consulta_cand_2016_PR.csv', **juntar_tse.kwargs)

In [ ]:
# puxar os números do protocolo da candidatura
amostra_pr = amostra_pr.merge(candidatos_pr, on='SQ_CANDIDATO')

In [ ]:
# disponibilizar na tela os números dos protocolos de candidatura
# amostra_pr.columns
amostra_pr[['NM_CANDIDATO', 'NM_UE_x', 'DS_CARGO', 'NR_PROTOCOLO_CANDIDATURA']]

In [ ]:
# criar link para raspar documentos
links = [
    f'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?' \
    f'nprot={nprot}&comboTribunal=pr' \
    for nprot in amostra_pr['NR_PROTOCOLO_CANDIDATURA'].to_list()
]

# disponibilizar os links para a gente
for link in links: 
    print(link)

## Baixar e processar os dados

In [ ]:
# usar módulo externo

In [ ]:
# importar meu pacote de análise
from tse import parser

In [ ]:
# construir path para os arquivos
decisões = [f'../dados/decisão_{i}.html' for i in range(0, 10)]

In [ ]:
# definir função para transformar os dados do sumário em tabela
def tabelar_sumário(decisão, transpor=True):
    sumário = pd.DataFrame(parser(decisão).parse_summary())
    if transpor: 
        sumário = sumário.T
    return sumário

In [ ]:
# produzir tabela
tabelar_sumário(decisões[1])

In [ ]:
# criar banco de dados
sumários = pd.concat([tabelar_sumário(decisão, False) for decisão in decisões], ignore_index=True)
sumários

In [ ]:
# onde estão estes processos? nas seguintes zonas eleitorais
sumários['district']

## Parte 3: Processamento de Linguagem Natural (NLP)

In [ ]:
# importar pacotes
import spacy
from bs4 import BeautifulSoup

# criar ferramentas de análise de texto
nlp_en = spacy.load('en_core_web_sm')
nlp_pt = spacy.load('pt_core_news_sm')

### Exemplo Genérico

In [ ]:
# definir função que baixa uma reportagem de jornal
def baixar_reportagem():
    URL = 'https://www.nytimes.com/2020/05/12/well/family/coronavirus-children-covid-19.html'
    r = requests.get(URL)
    soup = BeautifulSoup(r.content)
    text = soup.find('section', attrs={'name': 'articleBody'}).get_text()
    text = re.sub(r'(”|“)(?![A-z])', r'\1 ', text)
    text = re.sub(r' +', ' ', text)
    return text

In [ ]:
# baixar reportagem do NYTimes
baixar_reportagem()

In [ ]:
# salvar texto
texto = baixar_reportagem()

In [ ]:
# processar o texto
doc_en = nlp_en(texto)

In [ ]:
# contar o número de frases
frases = list(doc_en.sents)
len(frases)

In [ ]:
# mostrar uma frase qualquer
frases[13]

In [ ]:
# definir função para investigar o conteúdo de uma frase
def investigar_frase(frase):
    resultado = [{'token': ent.text, 'tipo': ent.label_ , 'descrição': spacy.explain(ent.label_)} for ent in frase.ents]
    return pd.DataFrame(resultado)

In [ ]:
# testar reconhecimento do texto
print(f'{frases[13]}\n\n{investigar_frase(frases[13])}')

In [ ]:
# testar reconhecimento do texto
print(f'{frases[43]}\n\n{investigar_frase(frases[43])}')

### Exemplo Real

In [ ]:
# abrir uma coletânea de decisões judiciais dentre todas
decisão = pd.DataFrame(parser(decisões[1]).parse_details())
decisão

In [ ]:
# extrair texto da decisão
texto = decisão.loc[3, 'sbody']
texto

### Mostrar as entidades reconhecidas pelo software nos autos

In [ ]:
# processar o texto 
doc_pt = nlp_pt(texto)

In [ ]:
# transformar em frases
frases = list(doc_pt.sents)

In [ ]:
# imprimir as entidades reconhecidas pela nossa ferramenta
pd.concat([pd.DataFrame(investigar_frase(frase)) for frase in frases])

### Análise Sintática

In [ ]:
# fazer análise sintática do texto
frase = str(frases[-3])
frase = nlp_pt(frase)

In [ ]:
# interpretar a frase mais detalhadamente
pd.DataFrame([
    {'token': token.text, 'classe gramatical': token.pos_, 'descrição': spacy.explain(token.pos_)}
    for token in frase
])

In [ ]:
spacy.displacy.render(frase, jupyter=True, style='dep', options={'compact': True})

### Destacar as entidades no texto

In [ ]:
# mostrar as entidades na frase
colors = {'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)'}
options = {'ents': 'ORG', 'colors': colors}
spacy.displacy.render(frase, jupyter=True, style='ent', options=options)